In [6]:
import warnings 
warnings.filterwarnings("ignore")

#### 이전에 feature engineering한 타이타닉 데이터 불러오기
 - onehot encoding 작업 완료
 - 처음부터 891개의 행까지만 훈련 데이터 (label 존재)
 - 그 이후는 테스트 데이터(label 미존재)

In [16]:
import pickle 


with open("titanic_step3_feature_engineering.pickle", "rb") as pickle_file:
    onehot_df = pickle.load(pickle_file)

with open("titanic_step3_feature_engineering_y.pickle", "rb") as pickle_file_y:
    y_train = pickle.load(pickle_file_y)

ntrain = 891
X_train,X_test = onehot_df[:ntrain], onehot_df[ntrain:]

print(f"훈련 데이터의 개수: {X_train.shape[0]}")

훈련 데이터의 개수: 891


In [17]:
X_train.head()

,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Age_0,Age_1,Age_2,Age_3,Age_4,...,HighSurvival_0,HighSurvival_1,HighSurvival_2,HighSurvival_3,HighSurvival_4,HighSurvival_5,HighSurvival_6,LowSurvival_0,LowSurvival_1,LowSurvival_2
0,False,False,True,True,False,False,False,False,True,False,...,True,False,False,False,False,False,False,True,False,False
1,True,False,False,False,True,False,False,False,False,True,...,False,False,True,False,False,False,False,True,False,False
2,False,False,True,False,True,False,False,False,True,False,...,True,False,False,False,False,False,False,True,False,False
3,True,False,False,False,True,False,False,False,False,True,...,False,False,True,False,False,False,False,True,False,False
4,False,False,True,True,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,False,True


#### label

In [10]:
y_train.value_counts()

Survived
0.0         549
1.0         342
Name: count, dtype: int64

#### K-Fold 실습

In [18]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [19]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=13)

## fold의 개수를 10개, 매 fold를 나눌 때 마다 데이터를 섞음
kfold = KFold(n_splits=10, shuffle=True, random_state=2025)

#### 각 fold에 의해 도출된 정확도의 평균을 냄

In [21]:
score = cross_val_score(knn,X_train, y_train, n_jobs=-1, cv=kfold, scoring="accuracy")
score

array([0.81111111, 0.78651685, 0.85393258, 0.84269663, 0.79775281,
       0.8988764 , 0.85393258, 0.85393258, 0.79775281, 0.79775281])

In [22]:
import numpy as np 
np.mean(score) * 100

np.float64(82.94257178526841)

#### Stratified K-Fold

In [24]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

iris_data = load_iris()


In [25]:
iris_df = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)

#### iris 데이터의 target

In [30]:
iris_df["species"] = iris_data.target
iris_df["species"].value_counts()

species
0    50
1    50
2    50
Name: count, dtype: int64

In [31]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


#### Stratified K-Fold의 데이터가 어떻게 분배되는지 확인
- 실제로 각각의 레이블의 비율에 맞게 균등하게 train 및 test dataset이 분배됨

In [32]:
skfold = StratifiedKFold(n_splits=3)

for train_index, test_index in skfold.split(iris_df, iris_df["species"]):
    trainset = iris_df["species"].iloc[train_index]
    testset = iris_df["species"].iloc[test_index]

    print("train set\n", trainset.value_counts())
    print("test set\n", testset.value_counts())

train set
 species
2    34
0    33
1    33
Name: count, dtype: int64
test set
 species
0    17
1    17
2    16
Name: count, dtype: int64
train set
 species
1    34
0    33
2    33
Name: count, dtype: int64
test set
 species
0    17
2    17
1    16
Name: count, dtype: int64
train set
 species
0    34
1    33
2    33
Name: count, dtype: int64
test set
 species
1    17
2    17
0    16
Name: count, dtype: int64


#### K-Fold의 경우 train dataset에는 1과 2 레이블인 데이터, test datset에는 0이 레이블인 데이터만 분배됨

In [33]:
kfold = KFold(n_splits=3)

for train_index, test_index in kfold.split(iris_df):
    trainset = iris_df["species"].iloc[train_index]
    testset = iris_df["species"].iloc[test_index]

    print("train set\n", trainset.value_counts())
    print("test set\n", testset.value_counts())

train set
 species
1    50
2    50
Name: count, dtype: int64
test set
 species
0    50
Name: count, dtype: int64
train set
 species
0    50
2    50
Name: count, dtype: int64
test set
 species
1    50
Name: count, dtype: int64
train set
 species
0    50
1    50
Name: count, dtype: int64
test set
 species
2    50
Name: count, dtype: int64
